In [ ]:
# The data used in this assignment can be found at https://www.kaggle.com/datasets/4e614ec846ab778f6a2ff166232d5a65f5e6786b4f5781690588bd2cccd71cb6?resource=download
# I used openly available online data instead of creating my own for this assignment.
# Thanks to TADASHINAGAO for compiling and sharing.

In [1]:
import logging
import os
import yaml
import pandas as pd
import re

def read_config_file(filepath):
    with open(filepath, "r") as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
                logging.error(exc)

def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

In [10]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")
config_data

{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'custom_1988_2020',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['ym_year_month',
  'exp_imp_export_1_import_2',
  'hs9_hs_code',
  'customs',
  'country',
  'q1',
  'q2_quantity',
  'value_in_thousands_of_yen']}

In [3]:
import pandas as pd
# read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file, config_data['inbound_delimiter'])
# Add column names as the data csv did not contain them
df.columns = ['ym(Year + month)',
  'exp_imp(export:1,import:2)',
  'hs9(HS code)',
  'Customs', 
  'Country',
  'Q1',
  'Q2(quantity)',
  'Value(in thousands of yen)']
df.head()

C:\Users\Nolan\AppData\Local\Temp\ipykernel_16688\1142813764.py:6: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv(source_file, config_data['inbound_delimiter'])


,ym(Year + month),"exp_imp(export:1,import:2)",hs9(HS code),Customs,Country,Q1,Q2(quantity),Value(in thousands of yen)
0,198801,1,103,100,120991000,0,1590,4154
1,198801,1,103,100,210390900,0,4500,2565
2,198801,1,103,100,220890200,0,3000,757
3,198801,1,103,100,240220000,0,26000,40668
4,198801,1,103,100,250410000,0,5,8070


In [11]:
#validate the header of the file
util.col_header_val(df, config_data)

column name and column length validation passed


1

In [12]:
print("columns of files are:", df.columns)
print("columns of YAML are:", config_data['columns'])

columns of files are: Index(['ym_year_month', 'exp_imp_export_1_import_2', 'hs9_hs_code', 'customs',
       'country', 'q1', 'q2_quantity', 'value_in_thousands_of_yen'],
      dtype='object')
columns of YAML are: ['ym_year_month', 'exp_imp_export_1_import_2', 'hs9_hs_code', 'customs', 'country', 'q1', 'q2_quantity', 'value_in_thousands_of_yen']


In [25]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine
    print(f'Total number of rows: {df.shape[0]}')
    print(f'Total number of columns: {df.shape[1]}')
    size_bytes = os.stat(source_file).st_size
    print(f'file size: {size_bytes}')
    
    df.to_csv(r'F:\Users\Nolan\Virtual Internship\internship_wk6\custom_1988_2020.txt.gz', index=None, sep='|', mode='a')

column name and column length validation passed
col validation passed
Total number of rows: 113607321
Total number of columns: 8
file size: 4544707885


KeyboardInterrupt: 